# A

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

#variables
dots = [(.5, 1.4), (2.3, 1.9), (2.9, 3.2)]

#functions
def get_error_curve(slope = .64):
    x_values = np.linspace(0, 2)
    y_values = []

    for x in x_values:
        y = 0

        for current_dot in dots:
            dot_x = current_dot[0]
            dot_y = current_dot[1]
            y += (dot_y - (x + (slope * dot_x))) ** 2
        
        y_values.append(y)
    
    return (x_values, y_values)


def get_gradient_line(intercept, slope):
    y_values = []

    for x in np.linspace(0, 4):
        y = intercept + (slope * x)
        y_values.append(y)
    
    return y_values


def get_tan(intercept, derivative, slope = .64):
    y_values = []

    y0 = 0 #error

    for current_dot in dots:
        dot_x = current_dot[0]
        dot_y = current_dot[1]
        y0 += (dot_y - (intercept + (slope * dot_x))) ** 2


    for x0 in graph_x_list:
        y = y0 + derivative * (x0 - intercept)
        y_values.append(y)
    
    return (y0, y_values)


def intercept_derivative_calculator(intercept, slope):
    derivative = 0
    for current_dot in dots:
        x = current_dot[0]
        y = current_dot[1]
        derivative += -2 * (y - (intercept + (slope * x)))
    
    return derivative


def gradient_descend(learning_rate = .1, step_size = 0, slope = .64, intercept = 0, minimum_step_size = .001, max_number_of_steps = 1000):
    animation_history = []
    tangent_list = []
    gradient_list = []

    step_count = 0
    while abs(step_size) > minimum_step_size or step_count == 0:
        current_history = []

        if step_count >= max_number_of_steps:
            break
        else:
            step_count += 1

        #getting the derivative
        intercept_derivative = intercept_derivative_calculator(intercept, slope)

        #getting the tangent
        current_error, current_tangent = get_tan(intercept, intercept_derivative, slope)
        current_history.append(current_error) #current error

        #updating the step size
        step_size = intercept_derivative * learning_rate
        current_history.append(step_size) #step size
        
        #updating he intercept
        current_history.append(intercept) #old intercept
        intercept -= step_size
        current_history.append(intercept) #new intercept

        #updating the history
        animation_history.append(current_history) #[error, step_size, current_intercept, new_intercept] history
        tangent_list.append(current_tangent)
        gradient_list.append(get_gradient_line(intercept, slope))
    
    return (step_count, animation_history, tangent_list, gradient_list)


def create_animation(step_count, animation_history, tangent_list, gradient_list):
    #error graph
    error_fig, error_graph = plt.subplots(figsize=(7, 7))

    error_graph.set_xlim(0, 2)
    error_graph.set_ylim(0, 4)
    error_graph.set_xticks([i * .25 for i in range(0, 9)])
    error_graph.set_yticks([i * .25 for i in range(0, 17)])
    error_graph.grid(True)
    error_graph.set_xlabel("Intercept", fontsize=12)
    error_graph.set_ylabel("Error", fontsize=12)
    error_graph.plot(graph_x_list, curve_y_list)
    plt.close()

    tangent_point, = error_graph.plot([], [], 'o', color='blue')
    tangent_line, = error_graph.plot([], [], '-', color='red')
    error_text = error_graph.text(
        0.025, 0.975, "", transform=error_graph.transAxes,
        fontsize=12, va='top'
    )

    def update_curve_animation(frame):
        history = animation_history[frame]
        error, step_size, current_intercept, new_intercept = history

        error_text.set_text(f"Step Size: {step_size:.2f}\nOld Intercept: {current_intercept:.2f}\nNew Intercept: {new_intercept:.2f}")


        tangent_point.set_data([current_intercept], [error])
        tangent_line.set_data(graph_x_list, tangent_list[frame])

        return tangent_point, tangent_line

    error_animation = FuncAnimation(error_fig, update_curve_animation, frames=step_count, interval=100, blit=True)

    #gradient graph
    gradient_fig, gradient_graph = plt.subplots(figsize=(7, 7))

    gradient_graph.set_xlim(0, 4)
    gradient_graph.set_ylim(0, 4)
    gradient_graph.set_xticks([i * .25 for i in range(0, 17)])
    gradient_graph.set_yticks([i * .25 for i in range(0, 17)])
    gradient_graph.grid(True)
    gradient_graph.set_xlabel("Weight", fontsize=12)
    gradient_graph.set_ylabel("Height", fontsize=12)
    plt.close()

    gradient_points, = gradient_graph.plot([], [], 'o', color='blue')
    gradient_line, = gradient_graph.plot([], [], '-', color='red')

    def update_line_animation(frame):
        gradient_points.set_data([dot[0] for dot in dots], [dot[1] for dot in dots])
        gradient_line.set_data(np.linspace(0, 4), gradient_list[frame])

        return gradient_points, gradient_line

    gradient_animation = FuncAnimation(gradient_fig, update_line_animation, frames=step_count, interval=100, blit=True)

    return [error_animation, gradient_animation]


#program
(graph_x_list, curve_y_list) = get_error_curve(.64)

#plot
error_anim1, gradient_anim1 = create_animation(*gradient_descend(learning_rate=.1))
error_anim2, gradient_anim2 = create_animation(*gradient_descend(learning_rate=.2))

anim_list = [error_anim1, gradient_anim1, error_anim2, gradient_anim2]
html = "".join(anim.to_jshtml() for anim in anim_list)
HTML(html)